# Opal Masterclass mit Maschine Learning

### 1. Vorbereitung

In diesem Block werden verschiedene Funktionen und Pakete geladen. Das musst du dir nicht weiter anschauen.

In [ ]:
import os, platform
if "opal_mc_ml_helpers.py" in os.listdir():
    print("Helpers schon vorhanden")
else:
    if platform.system() == "Linux":
        print("Helpers nicht vorhanden, starte Download...", end="")
        os.system("wget -O opal_mc_ml_helpers.py \"https://raw.githubusercontent.com/ntiltmann/opal-mc-ml/main/opal_mc_ml_helpers.py\"")
        print(" abgeschlossen!")
    elif platform.system() == "Darwin":
        print("Helpers nicht vorhanden, starte Download...", end="")
        os.system("curl -o opal_mc_ml_helpers.py \"https://raw.githubusercontent.com/ntiltmann/opal-mc-ml/main/opal_mc_ml_helpers.py\"")
        print(" abgeschlossen!")
    else:
        raise Exception("Kein Automatischer Download unter Windows möglich. Bitte alle Dateien manuell herunterladen und entpacken.")
from opal_mc_ml_helpers import *
check_files()

Die Funktion `load_events()` lädt alle Ereignisse und gibt sie als Liste aus. In der nächsten Zeile werden also alle Events in eine Liste mit dem Namen `eventliste` geladen.

In [ ]:
eventliste = load_events()

Da Neuronale Netzwerke mit mehr Trainingsdaten besser funktionieren, wurden aus diesen Ereignissen durch Kombination neue Ereignisse generiert, die für das Training des Neuronalen Netzwerks verwendet werden können.
Die Funktion `load_training_events()` lädt alle dieser Ereignisse und gibt sie wieder als eine Liste aus. In der nächsten Zeile werden also alle diese Ereignisse in eine Liste mit dem Namen `trainingsliste` geladen.

In [ ]:
trainingsliste = load_training_events()

Mit `eventliste[x]` kann auf das `x+1`-ste Ereignis zugegriffen werden. Die Methode `.show_image()` gibt das Ereignis grafisch aus. Dieser Schritt hat keine Auswirkungen auf das Neuronale Netz o.ä sondern dient nur der Visualisierung. 

In [ ]:
eventliste[540].show_image(show_category=True)

### 2. Übersichtstabelle anlegen

Es wird ein Objekt der Klasse `Overview` erstellt. Damit können später Übersichten der Verzweigungsverhältnisse angezeigt werden.

Mit `overview.add_entry()` kann eine bestimmte Eventliste der Übersicht hinzugefügt werden. Der erste Parameter gibt an, wie die Spalte beschriftet sein soll. Danach folgt die Eventliste selbst.

Mit `overview.show()` kann dann die Übersicht angezeigt werden.

In [ ]:
overview = Overview()
overview.add_entry("Roh", eventliste)
overview.show()

### 3. Training & Validierung

Als nächstes werden die Bilddateien in zwei Datensätze aufgeteilt. Einen für Training und Validierung (Abkürzung `tv`) und einen für den anschließenden Test des Neuronalen Netzes. Wähle an dieser Stelle das Verhältnis selber. Trage dafür eine Zahl zwischen `0`und `1` ein. Diese Zahl gibt an wie viel Prozent der Bilddateien in den Datensatz für Training und Validierung hinzugefügt werden. 

**Beispiel:** eventliste_tv, eventliste_test = split_events_random_in_category(eventliste, fraction_first_block=0.1) bedeutet, dass 10% jeder Eventkategorie der Bilddateien zufällig in den Datensatz für Training und Validierung (`eventliste_tv`) hinzugefügt werden und 90% für den anschließenden Test (`eventliste_test`) genutzt werden.

**Hinweis:** Bedenke, dass eine gute Datenbasis für Training und Validierung gebraucht wird, aber die für Training und Validierung verwendeten Daten bekannt sein müssen. Werden sehr viele Daten für Training und Validierung genutzt, bleiben nicht mehr ausreichend Daten für die eigentliche Datenauswertung mit dem Neuronalen Netz.   

In [ ]:
eventliste_tv, eventliste_test = split_events_random_in_category(eventliste, fraction_first_block=)

Die Übersicht wird aktualisiert und ausgegeben.

In [ ]:
overview.add_entry("TV", eventliste_tv)
overview.add_entry("Test", eventliste_test)
overview.show()

Im nächsten Schritt folgt die Data Augementation, das bedeutet, dass die `tv`-Daten mit Hilfe der `trainingsliste` vervielfältigt werden, damit das Training einfacher wird. Wähle zunächst für alle vier Faktoren (`faktor_q`, `faktor_e`, `faktor_m`, `faktor_t`) einen ganzzahligen Wert zwischen `1`und `5` mit dem die Daten vervielfältigt werden. Später kann es sinnvoll sein, diese Werte teilweise nochmal zu verändern. Grundsätzlich sind dann für `faktor_e`, `faktor_m`, `faktor_t` auch größere Werte als `5`. Das Maximum hängt von der Anzahl der Ereignisse in `eventliste_tv` ab und liegt bei `2n`, wobei n die Anzahl Ereignisse in der Kategorie ist.

In [ ]:
faktor_q =         # Quarks
faktor_e =         # Elektronen
faktor_m =         # Myonen 
faktor_t =         # Tauonen
eventliste_tv_vermehrt = augment_events(eventliste_tv, trainingsliste, [faktor_q, faktor_e, faktor_m, faktor_t])

Nun werden die `tv`-Daten in Training (`t`) und Validierung (`v`) unterteilt.

In [ ]:
eventliste_training, eventliste_validierung = split_events_random_in_category(eventliste_tv_vermehrt, fraction_first_block=0.8)

Die Übersicht wird aktualisiert und ausgegeben.

In [ ]:
overview.add_entry("TV verm", eventliste_tv_vermehrt)
overview.add_entry("Train", eventliste_training)
overview.add_entry("Vali", eventliste_validierung)
overview.show()

### 4. Modell für das Neuronale Netz anlegen 

Die Trainingsdaten sind jetzt passend vorbereitet. Im nächsten Block wird ein Modell zum Lernen erstellt. Die Struktur wird einmal ausgegeben. In den letzten beiden Zeilen werden die Trainings- und Validierungsdaten geladen.

In [ ]:
modell = MLModel()
modell.load_structure_default()
modell.show_structure()
modell.load_training_eventlist(eventliste_training)
modell.load_validation_eventlist(eventliste_validierung)

### 5. Training des Modells

Mit `train()` wird das Training des Modells gestartet. `count_epochs` gibt an, wie oft über alle Trainings- und Validierungsdaten gelaufen wird. Bei diesen Daten sind häufig Werte zwischen 2 und 8 sinnvoll. Probiere aus, welche Unterschiede du beobachten kannst. 

**Achtung:** Um Unterschiede zwischen der Anzahl an Trainingsrunden beobachten zu können, muss auch die Zelle vorher, in der das Modell für das Neuronale Netz angelegt wurde, erneut ausgeführt werden.

In [ ]:
modell.train(count_epochs=)

Die Lernkurve zeigt an, wie die Treffergenauigkeit sich im Laufe des Trainings verändert. Dabei wird die Treffergenauigkeit in Trainingsdaten und Validierungsdaten getrennt aufgetragen.

In [ ]:
modell.show_learning_curve()

### 6. Vorhersage der OPAL-Daten mit dem Neuronalen Netz

Mit dem trainierten Modell können wir nun die Kategorien der Testdaten vorhersagen. Dabei wird die schon zugewiesene Kategorie der Testdaten ignoriert. 

In [ ]:
eventliste_vorhersage = modell.predict(eventliste_test)

Die Confusion Matrix ist ein Indikator dafür, wie gut die Vorhersage mit dem Modell funktioniert. Ein perfektes Modell hätte nur Einträge auf der Diagonalen: Jede Kategorie wird vom Modell so erkannt, wie sie vorher schon dem Bild zugeordnet war. Tatsächlich treten häufig einige falsche Vorhersagen auf. 

In [ ]:
show_confusion_matrix(eventliste_test, eventliste_vorhersage)

Hier wird die Übersicht über die Verzweigungsverhältnisse noch um eine Spalte mit den vorhergesagten Daten erweitert.

In [ ]:
overview.add_entry("Vorh", eventliste_vorhersage)
overview.show()

Man kann sich nun noch anschauen, welche Bilder falsch vorhergesagt wurden. Gelegentlich ist eine Einteilung auch für Menschen nicht eindeutig möglich.

In [ ]:
show_false_predictions(eventliste_test, eventliste_vorhersage, count=5)